In [ ]:
# Energy Efficiency Prediction
# Predict building heating load using the UCI Energy Efficiency dataset.

# 1. Install and import libraries
!pip install numpy pandas matplotlib seaborn scikit-learn openpyxl


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
# 2. Load dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx'
df = pd.read_excel(url)
df.columns = [
    'RelativeCompactness', 'SurfaceArea', 'WallArea', 'RoofArea',
    'OverallHeight', 'Orientation', 'GlazingArea', 'GlazingAreaDistribution',
    'HeatingLoad', 'CoolingLoad'
]


In [8]:
# Count of NaN values per column
nan_counts = df.isna().sum()

# Count of duplicate rows
duplicate_count = df.duplicated().sum()

print("NaN Values per Column:\n", nan_counts)
print("\nTotal Duplicate Rows:", duplicate_count)

NaN Values per Column:
 RelativeCompactness        0
SurfaceArea                0
WallArea                   0
RoofArea                   0
OverallHeight              0
Orientation                0
GlazingArea                0
GlazingAreaDistribution    0
HeatingLoad                0
CoolingLoad                0
dtype: int64

Total Duplicate Rows: 0


In [3]:
# 3. Exploratory Data Analysis
default_figsize = (10, 8)
print(df.head())
print(df.info())
print(df.describe())

plt.figure(figsize=default_figsize)
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Feature Correlation Matrix')
plt.show()

   RelativeCompactness  SurfaceArea  WallArea  RoofArea  OverallHeight  \
0                 0.98        514.5     294.0    110.25            7.0   
1                 0.98        514.5     294.0    110.25            7.0   
2                 0.98        514.5     294.0    110.25            7.0   
3                 0.98        514.5     294.0    110.25            7.0   
4                 0.90        563.5     318.5    122.50            7.0   

   Orientation  GlazingArea  GlazingAreaDistribution  HeatingLoad  CoolingLoad  
0            2          0.0                        0        15.55        21.33  
1            3          0.0                        0        15.55        21.33  
2            4          0.0                        0        15.55        21.33  
3            5          0.0                        0        15.55        21.33  
4            2          0.0                        0        20.84        28.28  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 7

In [4]:
# 4. Preprocessing
X = df.drop(['HeatingLoad', 'CoolingLoad'], axis=1)
y = df['HeatingLoad']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
# 5. Model training and evaluation
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.1),
    'Support Vector Regression': SVR(),
    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=100, random_state=42
    )
}

results = []
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    results.append({
        'Model': name,
        'MSE': mean_squared_error(y_test, y_pred),
        'RMSE': mean_squared_error(y_test, y_pred),
        'MAE': mean_absolute_error(y_test, y_pred),
        'R2': r2_score(y_test, y_pred)
    })

results_df = pd.DataFrame(results)
print(results_df)

                       Model       MSE      RMSE       MAE        R2
0          Linear Regression  9.153208  9.153208  2.182073  0.912185
1           Ridge Regression  9.213843  9.213843  2.190779  0.911603
2           Lasso Regression  9.938754  9.938754  2.294477  0.904648
3  Support Vector Regression  7.969841  7.969841  1.888664  0.923538
4          Gradient Boosting  0.265311  0.265311  0.386133  0.997455


In [7]:
# 6. Visualize comparison
plt.figure(figsize=(8, 6))
sns.barplot(data=results_df, x='Model', y='RMSE')
plt.title('Model RMSE Comparison')
plt.ylabel('RMSE')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 7. Conclusions
# Compare MSE, RMSE, MAE, and R2 to select the best-performing model.
